<a href="https://colab.research.google.com/github/aomazic/Project-E/blob/main/Veles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mixtral game master

### Dependencies setup

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Data processing

In [ ]:
def convert_data(row):
    csv_data = []  # Create an empty list for each data point

    # Add comments for each section
    csv_data.append("<s>[INST] The following information represents a non-playable character in a game. Use this information to simulate a response from the character. The character can perform the following actions: goTo, eat, drink, speak, transfer. Each action has specific details associated with it, which are provided in the 'actionDetail' field.\n Response formats are consistant : response:(Character response) action : {'type': (type), 'actionDetail': {'source': (source)), 'target': (target)}}")
    csv_data.append("###character: ")
    csv_data.append(row["character"])
    csv_data.append("###question: ")
    csv_data.append(row["question"])
    csv_data.append("###memory: ")
    csv_data.append(row["memory"])
    csv_data.append("[/INST]\n")

    # Add response and action
    csv_data.append("###response: ")
    csv_data.append(row["response"])
    csv_data.append("###action: ")
    csv_data.append(row["action"])
    csv_data.append("</s>")

    formatted_data = " ".join(csv_data)

    return formatted_data

In [ ]:
def merge_columns(dataset):
    dataset["prediction"] = convert_data(dataset)
    return dataset

In [ ]:
import json
from datasets import Dataset
import pandas as pd

# Load your JSON data
with open("dataset.json", "r") as file:
    json_data = json.load(file)

df = pd.DataFrame(json_data)

# Write the DataFrame to a CSV file
df.to_csv('formatted_dataset.csv', index=False)

# Load the CSV file as a Dataset
data = Dataset.from_csv('formatted_dataset.csv')

data = data.map(merge_columns)

print(data)
print(data['prediction'][0])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['character', 'question', 'memory', 'response', 'action', 'prediction'],
    num_rows: 20
})
<s>[INST] The following information represents a non-playable character in a game. Use this information to simulate a response from the character. The character can perform the following actions: goTo, eat, drink, speak, transfer. Each action has specific details associated with it, which are provided in the 'actionDetail' field.
 Response formats are consistant : response:(Character response) action : {'type': (type), 'actionDetail': {'source': (source)), 'target': (target)}} ###character:  {'name': 'Fiona', 'age': 29, 'interests': ['Farming', 'Botany', 'Cooking'], 'description': 'Fiona is a skilled farmer who tends to her crops with care and expertise.'} ###question:  what to do today ###memory:  [{'date_created': '2023-02-02 08:00:00', 'recency': 8, 'importance': 7, 'activity': 'Spotted Ivan at 2023-02-01 10:00:00 in the World'}, {'date_created': '2023-02-01 10:00:00'

## Model setup

In [ ]:
import torch
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

NVIDIA L4


In [ ]:
import torch.nn as nn
import bitsandbytes as bnb
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig)
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config = bnb_config, device_map='auto')

model.config.quantization_config.to_dict()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 '_load_in_8bit': False,
 '_load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16',
 'bnb_4bit_quant_storage': 'uint8',
 'load_in_4bit': True,
 'load_in_8bit': False}

## Model finetuning

### Freezing the original weights


In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)


### Setting up the LoRa Adapters

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3758886912 || trainable%: 0.18132346515244138


### Training

In [ ]:
import transformers
from datasets import load_dataset

data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)


tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=20,
        max_steps=40,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,2.124300
2,2.141100
3,2.081500
4,2.061100


KeyboardInterrupt: 

## Merge adapters and share on the 🤗 Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("aomazic/Mistral-7B-Instruct-v0.2-veles", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:836: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/aomazic/Mistral-7B-Instruct-v0.2-veles/commit/08997c2a55a41308b72831539af601ffd30ab1f7', commit_message='Upload model', commit_description='', oid='08997c2a55a41308b72831539af601ffd30ab1f7', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [ ]:
peft_model_id = "aomazic/Mistral-7B-Instruct-v0.2-veles"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print(config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=True, r=16, target_modules={'v_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


In [ ]:
peft_model_id = "aomazic/Mistral-7B-Instruct-v0.2-veles"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, quantization_config = bnb_config, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
model.save_pretrained("veles")

merged_model = model.merge_and_unload()
merged_model.save_pretrained("veles",safe_serialization=True, max_shard_size="2GB")

merged_model.push_to_hub("aomazic/Mistral-7B-Instruct-v0.2-veles", use_auth_token=True)

input_ids = torch.ones(1, 1, dtype=torch.long)
attention_mask = torch.ones(1, 1, dtype=torch.long)


OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

## Inference

In [ ]:
batch = tokenizer("###character:  {'name': 'Fiona', 'age': 29, 'interests': ['Farming', 'Botany', 'Cooking'], 'description': 'Fiona is a skilled farmer who tends to her crops with care and expertise.'} ###question:  what to do today ###memory:  [{'date_created': '2023-02-02 08:00:00', 'recency': 8, 'importance': 7, 'activity': 'Spotted Ivan at 2023-02-01 10:00:00 in the World'}, {'date_created': '2023-02-01 10:00:00', 'recency': 8, 'importance': 7, 'activity': 'Thirst level at 80% at time 2023-02-01 10:00:00'}, {'date_created': '2023-01-31 12:00:00', 'recency': 8, 'importance': 7, 'activity': 'Told Ivan: How is your day?'}, {'date_created': '2023-01-30 12:00:00', 'recency': 8, 'importance': 7, 'activity': 'Spotted bench at location guard tower'}] [/INST]", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = merged_model.generate(**batch, max_new_tokens=400)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1644: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 [INST] The following information represents a non-playable character in a game. Use this information to simulate a response from the character. The character can perform the following actions: goTo, eat, drink, speak, transfer. Each action has specific details associated with it, which are provided in the 'actionDetail' field.###character:  {'name': 'Fiona', 'age': 29, 'interests': ['Farming', 'Botany', 'Cooking'], 'description': 'Fiona is a skilled farmer who tends to her crops with care and expertise.'} ###question:  what to do today ###memory:  [{'date_created': '2023-02-02 08:00:00', 'recency': 8, 'importance': 7, 'activity': 'Spotted Ivan at 2023-02-01 10:00:00 in the World'}, {'date_created': '2023-02-01 10:00:00', 'recency': 8, 'importance': 7, 'activity': 'Thirst level at 80% at time 2023-02-01 10:00:00'}, {'date_created': '2023-01-31 12:00:00', 'recency': 8, 'importance': 7, 'activity': 'Told Ivan: How is your day?'}, {'date_created': '2023-01-30 12:00:00', 'recency': 8, 'i

## Convert to ONNX format

In [ ]:
!pip -qqq install optimum[exporters]
!pip -qqq install onnxruntime
!pip -qqq install optimum[onnxruntime]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
!optimum-cli export onnx --model aomazic/Mistral-7B-Instruct-v0.2-veles aomazic/Mistral-7B-Instruct-v0.2-veles_onnx/

2024-05-21 16:12:23.385079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 16:12:23.385146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 16:12:23.386764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 16:12:24.526538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Framework not specified. Using pt to export the model.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resum

In [ ]:
!cd /content/aomazic/Mistral-7B-Instruct-v0.2-veles_onnx
!git init
!git remote add origin

Reinitialized existing Git repository in /content/.git/
usage: git remote add [<options>] <name> <url>

    -f, --fetch           fetch the remote branches
    --tags                import all tags and associated objects when fetching
                          or do not fetch any tag at all (--no-tags)
    -t, --track <branch>  branch(es) to track
    -m, --master <branch>
                          master branch
    --mirror[=(push|fetch)]
                          set up remote as a mirror to push to or fetch from



### Test ONNX model

In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering
import torch

In [ ]:
# Local path to the directory where the ONNX model and tokenizer are stored
local_model_path = "/content/aomazic/Mistral-7B-Instruct-v0.2-veles_onnx"  # Update with your local model path

# Load the tokenizer and model from the local directory
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = ORTModelForQuestionAnswering.from_pretrained(local_model_path, from_transformers=True)

# Prepare the input
input_text = (
    "<s>[INST] The following information represents a non-playable character in a game. "
    "Use this information to simulate a response from the character. ###character: "
    "{'name': 'Elena', 'age': 25, 'interests': ['Magic', 'Archery', 'Herbology'], 'description': "
    "'Elena is a skilled mage who has a deep connection with nature.', 'morning_routine': "
    "['Elena begins her day by meditating in the forest and collecting magical herbs.'], "
    "'evening_routine': ['Elena practices her magic spells and experiments with potion-making under the moonlight.']} "
    "###question: what to do today ###memory: "
    "[{'date_created': '2023-02-02 08:00:00', 'recency': 0.8, 'importance': 0.7, 'relevance': 0.9, 'activity': "
    "'Gathering rare ingredients for a powerful potion'}, {'date_created': '2023-02-01 10:00:00', 'recency': 0.7, "
    "'importance': 0.6, 'relevance': 0.8, 'activity': 'Exploring ancient ruins to uncover lost magical artifacts'}, "
    "{'date_created': '2023-01-31 12:00:00', 'recency': 0.6, 'importance': 0.5, 'relevance': 0.7, 'activity': "
    "'Assisting villagers with magical ailments'}, {'date_created': '2023-01-30 14:00:00', 'recency': 0.5, 'importance': "
    "0.4, 'relevance': 0.6, 'activity': 'Attending a council meeting to discuss magical defenses for the village'}, "
    "{'date_created': '2023-01-29 16:00:00', 'recency': 0.4, 'importance': 0.3, 'relevance': 0.5, 'activity': "
    "'Training apprentices in the ways of magic'}][/INST]"
)

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="pt")

# Prepare model inputs
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Perform inference
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Print the outputs
print(outputs)

The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead


FileNotFoundError: Cannot determine framework from given checkpoint location. There should be a pytorch_model*.bin for PyTorch or tf_model*.h5 for TensorFlow.

In [ ]:
import onnxruntime as ort
# Local path to the ONNX model
local_model_path = "/content/aomazic/Mistral-7B-Instruct-v0.2-veles_onnx/model.onnx"

# Initialize an ONNX Runtime session
session = ort.InferenceSession(local_model_path)

# Print input names and shapes
for idx, input_meta in enumerate(session.get_inputs()):
    print(f"Input {idx}: name={input_meta.name}, shape={input_meta.shape}, type={input_meta.type}")

# Print output names and shapes
for idx, output_meta in enumerate(session.get_outputs()):
    print(f"Output {idx}: name={output_meta.name}, shape={output_meta.shape}, type={output_meta.type}")


Input 0: name=input_ids, shape=['batch_size', 'sequence_length'], type=tensor(int64)
Input 1: name=attention_mask, shape=['batch_size', 'past_sequence_length + 1'], type=tensor(int64)
Input 2: name=position_ids, shape=['batch_size', 'sequence_length'], type=tensor(int64)
Input 3: name=past_key_values.0.key, shape=['batch_size', 8, 'past_sequence_length', 128], type=tensor(float)
Input 4: name=past_key_values.0.value, shape=['batch_size', 8, 'past_sequence_length', 128], type=tensor(float)
Input 5: name=past_key_values.1.key, shape=['batch_size', 8, 'past_sequence_length', 128], type=tensor(float)
Input 6: name=past_key_values.1.value, shape=['batch_size', 8, 'past_sequence_length', 128], type=tensor(float)
Input 7: name=past_key_values.2.key, shape=['batch_size', 8, 'past_sequence_length', 128], type=tensor(float)
Input 8: name=past_key_values.2.value, shape=['batch_size', 8, 'past_sequence_length', 128], type=tensor(float)
Input 9: name=past_key_values.3.key, shape=['batch_size', 8, '